In [1]:
import pandas as pd
import numpy as np
#pd.options.display.max_columns=1000

In [2]:
from featurewiz import LazyTransformer, SuloRegressor, SuloClassifier
from featurewiz import FeatureWiz, cross_val_model_predictions
from featurewiz import print_regression_metrics, print_classification_metrics,print_sulo_accuracy

Imported lazytransform v1.11. 

Imported featurewiz 0.5.2. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True, scalers="std",
        		category_encoders="auto", add_missing=False, verbose=0. imbalanced=False,
        		ae_options={})
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [3]:
trainfile = 'c:/users/ram/documents/ram/data_sets/kaggle/diabetes.csv'
datapath = '../Ram/Data_Sets/'
filename = 'winequality.csv'
#filename = 'sales.csv'
filename = 'affairs.csv'
trainfile = datapath+filename
sep = ','
dft = pd.read_csv(trainfile,sep=sep)
dft.drop(['affair','affairs'],axis=1, inplace=True)
print(dft.shape)
dft.head()

(6366, 9)


,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair_multiclass
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4


In [4]:
target = 'affair_multiclass'
preds = [x for x in list(dft) if x not in [target]]
modeltype = 'Multi_Classification'
len(preds)

8

In [5]:
from featurewiz import StackingClassifier_Multi, get_class_distribution

In [6]:
from sklearn.model_selection import train_test_split
from featurewiz import FE_kmeans_resampler
if modeltype == 'Regression':
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, random_state=1,)
    #X_train_over, y_train_over = FE_kmeans_resampler(X_train, y_train, target, smote='',verbose=0)
    #print(X_train_over.shape, X_test.shape)
    #train, test = pd.concat([X_train_over, pd.Series(y_train_over,name=target)], axis=1), pd.concat([X_test, y_test], axis=1)
    train, test = train_test_split(dft, test_size=0.20, random_state=42)
else:
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, 
                                                    stratify=dft[target],
                                                    random_state=42)
    train, test = train_test_split(dft, test_size=0.20, random_state=42,
                                                    stratify=dft[target]
                                                   )
print(X_train.shape, X_test.shape)

(5092, 8) (1274, 8)


In [7]:
transform_target = True

In [8]:
fwiz = FeatureWiz(corr_limit=0.80, feature_engg='dae_add', category_encoders='auto', 
                 dask_xgboost_flag=False, nrows=None, verbose=2, skip_sulov=True,
                 skip_xgboost=False, transform_target=transform_target) 
X_train_selected, y_train = fwiz.fit_transform(X_train, y_train) 

featurewiz is given 0.8 as correlation limit...
    final list of feature engineering given: ['dae_add']
    final list of category encoders given: ['onehot', 'label']
Since Auto Encoders are selected for feature extraction,
    Recursive XGBoost is skipped...
DenoisingAutoEncoder()
    AE dictionary given: dict_items([])
    final list of scalers given: [minmax]
Loaded input data. Shape = (5092, 8)
Performing hyper param selection...
    defining a pipeline with MinMaxScaler and DAE
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Epoch 1/10
128/128 [==============================] - 2s 6ms/step - loss: 0.2109 - mse: 0.1736 - val_loss: 0.1019 - val_mse: 0.0712 - lr: 0.0100
Epoch 2/10
128/128 [==============================] - 0s 4ms/step - loss: 0.0897 - mse: 0.0653 - val_loss: 0.0882 - val_mse: 0.0696 - lr: 0.0100
Epoch 3/10
128/128 [==============================] - 1s 5ms/step - loss: 0.0691 - mse: 0.0552 - val_loss: 0.0778 - val_mse: 0.0682 - lr: 0.0100
Epoch 4/10
128/

In [9]:
print(X_train_selected.shape)
X_train_selected.head()

(5092, 10)


,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,ae_feature_1,ae_feature_2
389,0.75,1.000000,1.000000,0.181818,0.666667,0.272727,0.4,0.4,0.566451,0.379408
1810,0.00,1.000000,1.000000,0.545455,0.666667,1.000000,0.6,0.6,0.000000,0.035628
841,0.75,0.000000,0.088889,0.181818,0.000000,0.272727,0.4,0.2,3.068312,0.870748
2089,0.75,0.183673,0.088889,0.000000,0.666667,0.636364,0.6,1.0,2.749694,0.414009
1971,0.75,0.387755,0.244444,0.181818,0.666667,0.272727,0.4,0.6,2.152196,0.583464


In [10]:
X_test_selected = fwiz.transform(X_test) 

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (1274, 8)
    Beware! feature_engg will add 100s, if not 1000s of additional features to your dataset!
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (1274, 8)
Shape of transformed data due to auto encoder = (1274, 10)
Returning dataframe with 10 features 


In [11]:
if transform_target:
    y_test = fwiz.lazy.yformer.transform(y_test)

In [12]:
from featurewiz import IterativeBinaryClassifier, IterativeDoubleClassifier_XGB
from featurewiz import IterativeDoubleClassifier, get_class_distribution

In [13]:
from featurewiz.blagging import BlaggingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB

In [14]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, classification_report

# Updating the Random Forest Classifier with the corrected class weights
if modeltype == 'Regression':
    rf_classifier = RandomForestRegressor(random_state=42)
else:
    # Correctly computing class weights for the classes present in the training set
    class_weights_dict_corrected = get_class_distribution(y_train)
    #rf_classifier = RandomForestClassifier(class_weight=class_weights_dict_corrected, random_state=42)
    rf_classifier =  IterativeDoubleClassifier(
                                weights={1: 0.5, 2: 0.5} 
                               )


# Fitting the classifier on the training data
rf_classifier.fit(X_train_selected, y_train)

# Predicting on the test set
y_pred = rf_classifier.predict(X_test_selected)

if modeltype == 'Regression':
    print_regression_metrics(y_test, y_pred, verbose=1)
else:
    # Evaluating the classifier
    y_probas = rf_classifier.predict_proba(X_test_selected)
    print_classification_metrics(y_test, y_pred, y_probas, verbose=1)

Bal accu 16%
ROC AUC = 0.70
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      1050
           1       0.11      0.03      0.05        86
           2       0.17      0.04      0.07        48
           3       0.00      0.00      0.00        26
           4       0.13      0.06      0.08        35
           5       0.00      0.00      0.00        17
           6       0.00      0.00      0.00        12

    accuracy                           0.79      1274
   macro avg       0.18      0.16      0.16      1274
weighted avg       0.70      0.79      0.74      1274

final average balanced accuracy score = 0.16


In [15]:
disto

NameError: name 'disto' is not defined

In [ ]:
def cross_validation( x_training, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )

        # start and end date for validation
        validation_start_date = x_training['date'].max() - datetime.timedelta(days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta(days=(k-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # training and validation dataset
        # training
        xtraining = training.drop( ['date', 'sales'], axis=1 )
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop( ['date', 'sales'], axis=1 )
        yvalidation = validation['sales']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )
        
        # store performance of each kfold iteration
        mae_list.append( m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )
    return pd.DataFrame( {'Model Name': model_name,
    'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
    'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
    'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str )}, index=[0] )

def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )

def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )

def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    return pd.DataFrame( { 'Model Name': model_name,
                            'MAE': mae,
                            'MAPE': mape,
                            'RMSE': rmse }, index=[0] )

In [ ]:
# Example usage
# Create and train the classifier
clf = IterativeDoubleClassifier(base_classifier1=BlaggingClassifier(), 
                                base_classifier2=RandomForestClassifier(), 
                                weights={1: 0.5, 2: 0.5} 
                               )
clf.fit(X_train, y_train)

# Predict
predictions = clf.predict(X_test)
probas = clf.predict_proba(X_test)
print('done')

In [ ]:
if modeltype == 'Regression':
    print_regression_metrics(y_test, predictions)
else:
    print_classification_metrics(y_test, predictions, probas, verbose=1)

In [ ]:
from imbens.ensemble import AsymBoostClassifier, SelfPacedEnsembleClassifier, BalancedRandomForestClassifier
from imbens.ensemble import RUSBoostClassifier, KmeansSMOTEBoostClassifier, AdaCostClassifier
from featurewiz import BlaggingClassifier

In [ ]:
#list_models = [AsymBoostClassifier, SelfPacedEnsembleClassifier, BalancedRandomForestClassifier,
#              RUSBoostClassifier, KmeansSMOTEBoostClassifier, AdaCostClassifier]
list_models = [StackingClassifier_Multi]
#list_models = [BlaggingClassifier]

In [ ]:
import time
import pdb
for model in list_models:
    start_time = time.time()
    print('\n',str( model).split(".")[-1])
    stack = model()
    stack.fit(X_train, y_train)
    test_predictions = stack.predict(X_test)
    test_probas = stack.predict_proba(X_test)
    if modeltype == 'Regression':
        print_regression_metrics(y_test, test_predictions)
    else:
        print_classification_metrics(y_test, test_predictions, test_probas, verbose=1)
    print('Time taken = %0.0f seconds' %(time.time()-start_time))

In [ ]:
disto

In [ ]:
if modeltype == 'Regression':
    print_regression_metrics(y_test, test_predictions)
else:
    print_classification_metrics(y_test, test_predictions, test_probas, verbose=1)

In [ ]:
#enco = 'catboost'
enco = "auto"
feature_engg = [
        "groupby",
        "interactions",
        #"target"
    ]

In [ ]:
if modeltype == 'Regression':
    ### SuloRegressor works well for Regression problems ##
    stack.fit(X_train_over, y_train_over)
    test_predictions = model.predict(X_test)
else:
    ## this works well only for binary_classification problems. It is bad for Multi-class ##
    ### SMOTE works well only for binary-class but is bad for multi-class problems - avoid!
    test_predictions, test_probas  = cross_val_model_predictions(stack, train, test, 
                                                feature_engg=feature_engg, 
                                                feature_selection=True,
                                                targets=[target], 
                                                modeltype=modeltype,
                                                smote=False)

In [ ]:
if modeltype == 'Regression':
    print_regression_metrics(y_test, test_predictions)
else:
    print_classification_metrics(y_test, test_predictions, test_probas, verbose=1)

In [ ]:
disto

In [ ]:
#datapath = '../Ram/Data_Sets/'
datapath = '../Ram/Data_Sets/'
filename = 'affairs.csv'
trainfile = datapath+filename
sep = ','
dft = pd.read_csv(trainfile, sep=sep, nrows=10000)
dft.drop(['affair_multiclass','affairs'],axis=1, inplace=True)
print(dft.shape)
dft.head()

In [ ]:
fwiz = FeatureWiz(
    corr_limit=0.9,
    feature_engg=feature_engg,
    category_encoders=enco,
    add_missing=True,
    nrows=None,
    verbose=0,
    transform_target=True,
    scalers=None
)

In [ ]:
X_train_selected, y_train = fwiz.fit_transform(
    X=X_train,
    y=y_train,
)

X_test_selected  = fwiz.transform(X_test)

In [ ]:
### This transforms y_test alone without touching X_test. Nice trick!
if modeltype != 'Regression':
    y_test = fwiz.lazy.yformer.transform(y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
if modeltype == 'Regression':
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=99)
else:
    model = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=99)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

In [ ]:
from lazytransform import print_classification_metrics, print_regression_metrics
if modeltype == 'Regression':
    print_regression_metrics(y_test.values, y_pred)
else:
    y_proba = model.predict_proba(X_test_selected)
    print_classification_metrics(y_test.values, y_pred, y_proba, verbose=1)

In [ ]:
print_sulo_accuracy(y_test, y_pred, y_proba)

In [ ]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
if modeltype == 'Regression':
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=99)
    model = XGBRegressor(n_estimators=5000, n_jobs=-1, random_state=99)
else:
    model = RandomForestClassifier(n_estimators=500,  n_jobs=-1, random_state=99)
    model = XGBClassifier(n_estimators=5000, n_jobs=-1, random_state=99)
#help(model)

In [ ]:
from featurewiz import cross_val_model_predictions

In [ ]:
if modeltype == 'Regression':
    print_regression_metrics(y_test.values, test_predictions,verbose=1)
else:
    print_classification_metrics(y_test.values, test_predictions, test_probas,verbose=1)

In [ ]:
disto

In [ ]:
import time
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
start_time = time.time()

# Set the target variable
feats_list = {}
feats_df = pd.DataFrame(feats_list, columns=['Method','Features', 'Num_Features','Mean RMSE']).set_index('Method')

# Run featurewiz to select the best features
fwiz = FeatureWiz(feature_engg=['interactions', 'target'], corr_limit=0.9, verbose=0)

# Convert categorical columns to numeric
dfd = pd.get_dummies(dft, drop_first=True)

#### split into df and test so that we can set up a hold out set
df, test = train_test_split(dfd, test_size=0.05, random_state=99)

### select features by transforming train
df_transformed, df_target = fwiz.fit_transform(df.drop(target, axis=1), df[target])
features = fwiz.features
test_transformed = test[features]

# Display the selected features
feats_df.loc['featurewiz', 'Features'] = f"{features}"
feats_df.loc['featurewiz', 'Num_Features'] = len(features)
model = RandomForestRegressor(random_state=99)
sco = cross_val_score(model, df_transformed, df_target, cv=5, n_jobs=-1, verbose=0, scoring='neg_root_mean_squared_error')*(-1)
feats_df.loc['featurewiz','Mean RMSE'] = sco.mean()
model = RandomForestRegressor(random_state=99)
model.fit(df_transformed, df_target)
sco2 = np.sqrt(mean_squared_error(test[target], model.predict(test_transformed)))
feats_df.loc['featurewiz','Test RMSE'] = sco2.mean()
print("Time taken = %0.0f seconds" %(time.time()-start_time))

In [ ]:
feats_df

In [ ]:
disto

# First we test the model with all features (preds)

In [ ]:
from featurewiz import simple_XGBoost_model
y_preds = simple_XGBoost_model(train[preds], train[target], test[preds])
ypreds0 = y_preds[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
if modeltype != 'Regression':
    le = LabelEncoder()
    le.fit(train[target])
    ypreds0 = le.inverse_transform(ypreds0)

# Here is the performance with all features

In [ ]:
from lazytransform import print_regression_model_stats, print_sulo_accuracy
if modeltype == 'Regression':
    print_regression_model_stats(test[target], ypreds0, verbose=1)
else:
    print_sulo_accuracy(test[target], ypreds0,verbose=1)

# next we train a similar model with featurewiz features

In [ ]:
from featurewiz import simple_XGBoost_model
y_preds = simple_XGBoost_model(train[feats], train[target], test[feats])
ypreds1 = y_preds[0]

# And now the results with featurewiz features

In [ ]:
from sklearn.preprocessing import LabelEncoder
if modeltype != 'Regression':
    le = LabelEncoder()
    le.fit(train[target])
    ypreds1 = le.inverse_transform(ypreds1)

In [ ]:
from lazytransform import print_regression_model_stats, print_sulo_accuracy
if modeltype == 'Regression':
    print_regression_model_stats(test[target], ypreds1, verbose=1)
else:
    print_sulo_accuracy(test[target], ypreds1,verbose=1)

# Results with featurewiz selected features are better!